In [1]:
from arcgis.gis import GIS
from arcgis.graph import KnowledgeGraph
from uuid import UUID
from pprint import pprint

### Connect

In [6]:
username = "tkinlaw_ent"
password = "Esri.4.GIS"
#url = "https://adsrv2019.ad.local/server/rest/services/Hosted/v8/KnowledgeGraphServer"
#gis = GIS(r"https://adsrv2019.ad.local/portal", username, password)
gis = GIS(r"https://adsrv2019.ad.local/portal", client_id="5PEin6xagJkBNTnX", verify_cert=False)
knowledge_graph = KnowledgeGraph(url=url, gis=gis)
knowledge_graph

Setting `verify_cert` to False is a security risk, use at your own risk.


Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://adsrv2019.ad.local/portal/sharing/rest/oauth2/authorize?response_type=code&client_id=5PEin6xagJkBNTnX&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=COyckvi6BsExQamGYX4Br8f7BbKtdo&allow_verification=false


### Get list of distributor names to process

In [11]:
distributorNameQuery = """MATCH (p:ProcessingPlant)
WITH DISTINCT p.Distributors as d
RETURN d"""
x=knowledge_graph.query(distributorNameQuery)
y = []
distributorNames = []
for i in range(len(x)):
    y.extend(x[i])
for item in y:
    temp = item.split(", ")
    for s in temp:
        distributorNames.append(s)
distributorNames = list(set(distributorNames))
print(distributorNames)

['Distributor 10', 'Distributor 8', 'Distributor 22', 'Distributor 3', 'Distributor 19', 'Distributor 21', 'Distributor 6', 'Distributor 17', 'Distributor 1', 'Distributor 23', 'Distributor 13', 'Distributor 18', 'Distributor 14', 'Distributor 9', 'Distributor 15', 'Distributor 7', 'Distributor 4', 'Distributor 5', 'Distributor 20', 'Distributor 16', 'Distributor 12', 'Distributor 2', 'Distributor 11']


### Query

In [12]:
def submitQuery(distributorName):
    opQuery = """MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "{0}")
    MATCH (d:Distributor)
    WHERE d.DistributorName = "{0}"
    RETURN p, d""".format(distributorName)
    print(opQuery)

    queryResults = knowledge_graph.query(opQuery)
    return queryResults

### Given returned nodes, create relationships

In [13]:
def createRelationships(queryResults):
    countRels = 0
    for p, d in queryResults:
        print("Related nodes:")
        print(p['_properties']['ProcessingPlantName'])
        #print(p['_id'])
        #print(p['_properties']['Material'])
        print(d['_properties']['DistributorName'])
        print(d['_id'])
        #print(d['_properties']['Material'])
        #print()
        add_relationship = {
            "_objectType": "relationship",
            "_typeName": "SellsTo",
            "_originEntityId": UUID(f"{d['_id']}"), # Distributor ID
            "_destinationEntityId": UUID(f"{p['_id']}") # Plant ID
        }
        knowledge_graph.apply_edits(adds=[add_relationship])
        countRels += 1
        #print("EDITS MADE")
    print(f"All done! {countRels} relationships created.")

### Perform the edits

In [14]:
for distributor in distributorNames:
    print(f"Finding related Processing Plants for {distributor}")
    queryNodes = submitQuery(distributor)
    createRelationships(queryNodes) 
print("End of script.")

Finding related Processing Plants for Distributor 10
MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "Distributor 10")
    MATCH (d:Distributor)
    WHERE d.DistributorName = "Distributor 10"
    RETURN p, d
Related nodes:
Plant 5
Distributor 10
1ebc1197-40bb-4f63-b2a9-7d72ce02f65f
Related nodes:
Plant 6
Distributor 10
1ebc1197-40bb-4f63-b2a9-7d72ce02f65f
Related nodes:
Plant 7
Distributor 10
1ebc1197-40bb-4f63-b2a9-7d72ce02f65f
All done! 3 relationships created.
Finding related Processing Plants for Distributor 8
MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "Distributor 8")
    MATCH (d:Distributor)
    WHERE d.DistributorName = "Distributor 8"
    RETURN p, d
Related nodes:
Plant 1
Distributor 8
05e845a0-81bd-4957-9534-4e97c3079310
Related nodes:
Plant 3
Distributor 8
05e845a0-81bd-4957-9534-4e97c3079310
All done! 2 relationships created.
Finding related Processing Plants for Distributor 22
MATCH (p: